# IMPORT AND DATA LOADING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import necessari
import librosa
import numpy
import sys
import os
import glob
import itertools
import re
import pickle
import keras
import tensorflow as tf
import tensorflow.keras.models
import tensorflow.keras.backend as K
import keras.optimizers
from keras.models import Model
from tensorflow.keras.constraints import max_norm
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Dropout
from tqdm import tqdm
from itertools import groupby
from keras.utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import train_test_split

# costanti 
ALPHA = 0.75
N_MELS = 128
HOP_LENGTH = 512
N_FFT = 1024
POWER = 2.0
FRAME_NUMS = 313
FRAMES = 10
VAL = 0.05

# FEATURES EXTRACTION

# Loading da Google Drive
train_data = numpy.load("/content/drive/MyDrive/DCASE_DATA_EXTRACTED/train/training_valve.npy")
grouped_list_by_machine_id = pickle.load( open( "/content/drive/MyDrive/DCASE_DATA_EXTRACTED/train/training_valve_grouped_list.npy", "rb" ) )

In [ ]:
# GENERAZIONE DELLE LABELS
# One-hot encoding
label = []
choices = []
for i in range(0, len(grouped_list_by_machine_id)):
  for j in range(0, len(grouped_list_by_machine_id[i])):
    machine_id = grouped_list_by_machine_id[i][j].split('/')[7].split('_')[2]
    #print(grouped_list_by_machine_id[i][j].split('/')[7])
    random_choice = numpy.random.choice(["match", "non_match"], p = [ALPHA, 1-ALPHA]) 

    if machine_id == '00':
      if random_choice == "match":
        to_append = [1,0,0,0]
      else: 
        not_match_label = numpy.random.choice([1, 2, 3]) 
        if not_match_label == 1:
          to_append = [0,1,0,0]
        elif not_match_label == 2:
          to_append = [0,0,1,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == '02': 
      if random_choice == "match":
        to_append = [0,1,0,0]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,0,1,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == "04":
      if random_choice == "match":
        to_append = [0,0,1,0]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,1,0,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == "06":
      if random_choice == "match":
        to_append = [0,0,0,1]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,1,0,0]
        else: 
          to_append = [0,0,1,0]
    
    label.append(to_append) # Append della label associata a ciascuno spettrogramma
    choices.append(random_choice) # Append della choice utilizzata per associare la label
                                  # La choice sarà utile in fase di addestramento per capire che tipo di loss calcolare

# Trasformazione in numpy.array     
label = numpy.asarray(label)
choices = numpy.asarray(choices)
print(label.shape)
print(choices.shape)

(3291, 4)
(3291,)


In [ ]:
print(len(grouped_list_by_machine_id[0]))
print(len(grouped_list_by_machine_id[1]))
print(len(grouped_list_by_machine_id[2]))
print(len(grouped_list_by_machine_id[3]))

891
608
900
892


In [ ]:
# Estrazione spettrogrammi divisi per ID
id_00 = train_data[0:891]
label_00 = label[0:891]
choices_00 = choices[0:891]

id_02 = train_data[891:1499]
label_02 = label[891:1499]
choices_02 = choices[891:1499]

id_04 = train_data[1499:2399]
label_04 = label[1499:2399]
choices_04 = choices[1499:2399]

id_06 = train_data[2399:3291]
label_06 = label[2399:3291]
choices_06 = choices[2399:3291]

id_00_training, \
id_00_validation, \
label_00_train, \
label_00_validation, \
choices_00_train, \
choices_00_validation = train_test_split(id_00, label_00, choices_00, test_size=VAL, random_state=42)

id_02_training, \
id_02_validation, \
label_02_train, \
label_02_validation, \
choices_02_train, \
choices_02_validation = train_test_split(id_02, label_02, choices_02, test_size=VAL, random_state=42)

id_04_training, \
id_04_validation, \
label_04_train, \
label_04_validation, \
choices_04_train, \
choices_04_validation = train_test_split(id_04, label_04, choices_04, test_size=VAL, random_state=42)

id_06_training, \
id_06_validation, \
label_06_train, \
label_06_validation, \
choices_06_train, \
choices_06_validation = train_test_split(id_06, label_06, choices_06, test_size=VAL, random_state=42)

# Normalization ID_00
id_00_norm = numpy.empty_like(id_00_training)
mean_00 = numpy.mean(id_00_training)
std_00 = numpy.std(id_00_training)
id_00_norm = (id_00_training - mean_00) / (std_00)
id_00_norm_validation = (id_00_validation - mean_00) / (std_00)

# Normalization ID_02
id_02_norm = numpy.empty_like(id_02_training)
mean_02 = numpy.mean(id_02_training)
std_02 = numpy.std(id_02_training)
id_02_norm = (id_02_training - mean_02) / (std_02)
id_02_norm_validation = (id_02_validation - mean_02) / (std_02)

# Normalization ID_04
id_04_norm = numpy.empty_like(id_04_training)
mean_04 = numpy.mean(id_04_training)
std_04 = numpy.std(id_04_training)
id_04_norm = (id_04_training - mean_04) / (std_04)
id_04_norm_validation = (id_04_validation - mean_04) / (std_04)

# Normalization ID_06
id_06_norm = numpy.empty_like(id_06_training)
mean_06 = numpy.mean(id_06_training)
std_06 = numpy.std(id_06_training)
id_06_norm = (id_06_training - mean_06) / (std_06)
id_06_norm_validation = (id_06_validation - mean_06) / (std_06)

print("==== DATA ====")
total_training = numpy.concatenate([id_00_norm, id_02_norm, id_04_norm, id_06_norm])
print(total_training.shape)
total_validation = numpy.concatenate([id_00_norm_validation, id_02_norm_validation, id_04_norm_validation, id_06_norm_validation])
print(total_validation.shape)

print("==== LABELS ====")
total_training_label = numpy.concatenate([label_00_train, label_02_train, label_04_train, label_06_train])
print(total_training_label.shape)
total_validation_label = numpy.concatenate([label_00_validation, label_02_validation, label_04_validation, label_06_validation])
print(total_validation_label.shape)

print("==== CHOICES ====")
total_training_choices = numpy.concatenate([choices_00_train, choices_02_train, choices_04_train, choices_06_train])
print(total_training_choices.shape)
total_validation_choices = numpy.concatenate([choices_00_validation, choices_02_validation, choices_04_validation, choices_06_validation])
print(total_validation_choices.shape)

==== DATA ====
(3125, 128, 313)
(166, 128, 313)
==== LABELS ====
(3125, 4)
(166, 4)
==== CHOICES ====
(3125,)
(166,)


In [ ]:
training_aug = numpy.zeros((len(total_training)*16, 128, 32)) # Dataset utilizzato per il training
index = 0
for vector_array in total_training:
  i = 0
  while (i+32) <= 313:
    vector_i = numpy.zeros((128,32))
    for j in range(0,128):
      vector_i[j] = vector_array[j][i:i+32]
    training_aug[index] = vector_i
    index += 1
    i = i+18

validation_aug = numpy.zeros((len(total_validation)*16, 128, 32)) # Dataset utilizzato per il training
index = 0
for vector_array in total_validation:
  i = 0
  while (i+32) <= 313:
    vector_i = numpy.zeros((128,32))
    for j in range(0,128):
      vector_i[j] = vector_array[j][i:i+32]
    validation_aug[index] = vector_i
    index += 1
    i = i+18

training_aug_transpose = numpy.zeros((len(training_aug), 32, 128))
index = 0
for elem in training_aug:
  training_aug_transpose[index] = elem.T
  index += 1
print(training_aug_transpose.shape)

validation_aug_transpose = numpy.zeros((len(validation_aug), 32, 128))
index = 0
for elem in validation_aug:
  validation_aug_transpose[index] = elem.T
  index += 1
print(validation_aug_transpose.shape)


####### LABELS ######
# Associazione della label associata a ciascun spettrogramma a ciascuno dei frame estratto da esso.
training_labels = []
for elem in total_training_label:
  if numpy.array_equal(elem, numpy.asarray([1,0,0,0])) :
    for i in range(16):
      training_labels.append([1,0,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,1,0,0])):
    for i in range(16):
      training_labels.append([0,1,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,1,0])):
    for i in range(16):
      training_labels.append([0,0,1,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,0,1])):
    for i in range(16):
      training_labels.append([0,0,0,1])

validation_labels = []
for elem in total_validation_label:
  if numpy.array_equal(elem, numpy.asarray([1,0,0,0])) :
    for i in range(16):
      validation_labels.append([1,0,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,1,0,0])):
    for i in range(16):
      validation_labels.append([0,1,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,1,0])):
    for i in range(16):
      validation_labels.append([0,0,1,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,0,1])):
    for i in range(16):
      validation_labels.append([0,0,0,1])

training_labels = numpy.asarray(training_labels) # Dataset utilizzato per il training
validation_labels = numpy.asarray(validation_labels) # Dataset utilizzato per il training
#####################


####### CHOICES ######
# Associazione della choice associata a ciascun spettrogramma a ciascuno dei frame estratto da esso. 
training_choices = []
for elem in total_training_choices:
  if numpy.array_equal(elem, numpy.asarray("match")) :
    for i in range(16):
      training_choices.append("match")
  elif numpy.array_equal(elem, numpy.asarray("non_match")):
    for i in range(16):
      training_choices.append("non_match")

validation_choices = []
for elem in total_validation_choices:
  if numpy.array_equal(elem, numpy.asarray("match")) :
    for i in range(16):
      validation_choices.append("match")
  elif numpy.array_equal(elem, numpy.asarray("non_match")):
    for i in range(16):
      validation_choices.append("non_match")

training_choices = numpy.asarray(training_choices) # Dataset utilizzato per il training
validation_choices = numpy.asarray(validation_choices) # Dataset utilizzato per il training
######################

print("==== DATA ====")
print(training_aug_transpose.shape)
print(validation_aug_transpose.shape)

print("==== LABELS ====")
print(training_labels.shape)
print(validation_labels.shape)

print("==== CHOICES ====")
print(training_choices.shape)
print(validation_choices.shape)

(50000, 32, 128)
(2656, 32, 128)
==== DATA ====
(50000, 32, 128)
(2656, 32, 128)
==== LABELS ====
(50000, 4)
(2656, 4)
==== CHOICES ====
(50000,)
(2656,)


# KERAS MODEL DEFINITION

In [ ]:
timesteps = 32
num_features = 128

input_Spect = Input(shape = [timesteps, num_features])
input_Label = Input(shape = [4,])

x = LSTM(64, 
        batch_input_shape=(None, timesteps, num_features), 
        return_sequences=True, name='encoder_1', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(input_Spect)

x = LSTM(32, 
        return_sequences=True, name='encoder_2', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(x)

x = LSTM(16, 
        return_sequences=False, 
        name='encoder_3', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(x)

# Second Branch - Conditioning Feed Forward Neural Network
m = Dense(16)(input_Label)
m = Activation('sigmoid')(m)
q = Dense(16)(input_Label)

# Encoded Input Conditioning
m = Multiply()([x, m])
encoded_input_conditioned = Add()([q, m]) # Input da passare al decoder

x = RepeatVector(timesteps, name='encoder_decoder_bridge')(encoded_input_conditioned)

x = LSTM(16, 
        return_sequences=True, name='decoder_1', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(x)

x = LSTM(32, 
        return_sequences=True, name='decoder_2', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(x)

x = LSTM(64, 
        return_sequences=True, name='decoder_3', 
        kernel_constraint = max_norm(1), 
        recurrent_constraint = max_norm(1), 
        bias_constraint = max_norm(1))(x)

x = TimeDistributed(Dense(num_features))(x)

loss_tracker = keras.metrics.Mean(name="loss")
mse_metric = keras.metrics.MeanSquaredError(name="mse")

class CustomModel(tensorflow.keras.Model):
    @property
    def metrics(self):
        return [loss_tracker, mse_metric]

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self([x[0],x[1]], training=False)
        # Indici match
        match = tf.where ( tf.equal(x[2][:], "match") )
        # Dati match
        data_match = K.gather(y, match)
        # Separazione dei dati PREDETTI sulla base degli indici relativi a match/non_match
        # Dati match
        pred_match = K.gather(y_pred, match)

        # Update metrica
        mse_metric.update_state(data_match, pred_match)

        return {"mse": mse_metric.result()}
    
    def train_step(self, data):
          # Unpack the data. Its structure depends on your model and on what you pass to `fit()`.
          x, y = data

          # Vettore C utilizzato per il calcolo della loss in caso di non_match
          C = 5 
          # Valore di probabilità utilizzato come peso
          ALPHA = 0.75 

          # Indici match
          match = tf.where ( tf.equal(x[2][:], "match") )

          # Indici non_match
          not_match = tf.where ( tf.equal(x[2][:], "non_match") )

          # Dati match
          data_match = K.gather(y, match)

          with tf.GradientTape() as tape:
              y_pred = self([x[0],x[1]], training=True)  # Forward pass

              # Separazione dei dati PREDETTI sulla base degli indici relativi a match/non_match
              # Dati match
              pred_match = K.gather(y_pred, match)
              # Dati non match
              pred_not_match = K.gather(y_pred, not_match) 

              loss_m = K.mean(keras.losses.mean_squared_error(data_match, pred_match)) + 1e-6  # Calcolo Loss Match
              loss_nm = K.mean(keras.losses.mean_squared_error(C,pred_not_match)) + 1e-6     # Calcolo Loss Non_Match

              loss = ALPHA * loss_m + (1 - ALPHA) * loss_nm     # loss utilizzata per l'update dei pesi

          # Compute gradients
          trainable_vars = self.trainable_variables
          gradients = tape.gradient(loss, trainable_vars)

          # Update weights
          self.optimizer.apply_gradients(zip(gradients, trainable_vars))

          # Compute our own metrics
          loss_tracker.update_state(loss)
          mse_metric.update_state(y, y_pred)
          return {"loss": loss_tracker.result(), "mse": mse_metric.result()}

model = CustomModel(inputs=(input_Spect, input_Label), outputs = x)
model.compile(metrics=["mse"], optimizer = "adam")
model.summary()

Model: "custom_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 128)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
encoder_1 (LSTM)                (None, 32, 64)       49408       input_1[0][0]                    
__________________________________________________________________________________________________
encoder_2 (LSTM)                (None, 32, 32)       12416       encoder_1[0][0]                  
_______________________________________________________________________________________

In [ ]:
history = model.fit([training_aug_transpose, training_labels, training_choices],
                    training_aug_transpose, 
                    epochs=200, 
                    validation_data=([validation_aug_transpose, validation_labels, validation_choices], validation_aug_transpose), 
                    batch_size=512)

Epoch 1/200
98/98 [==============================] - 9s 51ms/step - loss: 0.3229 - mse: 6.4865 - val_mse: 1.1192
Epoch 2/200
98/98 [==============================] - 3s 30ms/step - loss: 0.3336 - mse: 6.4831 - val_mse: 0.9772
Epoch 3/200
98/98 [==============================] - 3s 29ms/step - loss: 0.3404 - mse: 6.4713 - val_mse: 1.2976
Epoch 4/200
98/98 [==============================] - 3s 30ms/step - loss: 0.2754 - mse: 6.5351 - val_mse: 1.0797
Epoch 5/200
98/98 [==============================] - 3s 29ms/step - loss: 0.2868 - mse: 6.5179 - val_mse: 1.1751
Epoch 6/200
98/98 [==============================] - 3s 28ms/step - loss: 0.2637 - mse: 6.5395 - val_mse: 1.3210
Epoch 7/200
98/98 [==============================] - 3s 28ms/step - loss: 0.3417 - mse: 6.4669 - val_mse: 1.2506
Epoch 8/200
98/98 [==============================] - 3s 28ms/step - loss: 0.2880 - mse: 6.5229 - val_mse: 1.4070
Epoch 9/200
98/98 [==============================] - 3s 29ms/step - loss: 0.2872 - mse: 6.5200 -

In [ ]:
model.save('/content/drive/MyDrive/models/valve/LSTM/model_valve.h5') 

In [ ]:
with open('/content/drive/MyDrive/models/valve/LSTM/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# TESTING

In [ ]:
import csv

def save_csv(save_file_path,
             save_data):
    with open(save_file_path, "w", newline="") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(save_data)


# load dataset
def select_dirs(path):
    dir_path = os.path.abspath(path)
    dirs = sorted(glob.glob(dir_path))
    return dirs

def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))

def file_list_generator(target_dir, dir_name="train", ext="wav"):
    print("target_dir : {}".format(target_dir))

    # generate training list
    training_list_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    files = sorted(glob.glob(training_list_path))
    if len(files) == 0:
      print("errore")
    return files


def file_to_vector_array(file_name, n_mels=64, n_fft=1024, hop_length=512, power=2.0):
    # 02 generate melspectrogram using librosa
    y, sr = file_load(file_name)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)

    # 03 convert melspectrogram to log mel energy
    log_mel_spectrogram = 20.0 / power * numpy.log10(mel_spectrogram + sys.float_info.epsilon)

    return log_mel_spectrogram

  
def list_to_vector_array(file_list, msg="calc...", n_mels=64, n_fft=1024, hop_length=512, power=2.0, frames=10):
    # iterate file_to_vector_array()
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, power=power)

        if idx == 0:
            dataset = numpy.zeros((len(file_list), n_mels, frames), float)
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array
    return dataset

def key_by_id(item):
  path_splitted = item.split("/")
  file_name = path_splitted[ len(path_splitted) - 1 ]
  file_name_splitted = file_name.split("_")
  machine_id = file_name_splitted = file_name_splitted[2]
  return machine_id

def get_machine_id_list_for_test(target_dir,
                                 dir_name="test",
                                 ext="wav"):

    # create test files
    dir_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    file_paths = sorted(glob.glob(dir_path))
    # extract id
    machine_id_list = sorted(list(set(itertools.chain.from_iterable(
        [re.findall('id_[0-9][0-9]', ext_id) for ext_id in file_paths]))))
    return machine_id_list

def test_file_list_generator(target_dir,
                             id_name,
                             dir_name="test",
                             prefix_normal="normal",
                             prefix_anomaly="anomaly",
                             ext="wav"):
  
    print("target_dir : {}".format(target_dir+"_"+id_name))

    normal_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_normal}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                 dir_name=dir_name,
                                                                                 prefix_normal=prefix_normal,
                                                                                 id_name=id_name,
                                                                                 ext=ext)))
    normal_labels = numpy.zeros(len(normal_files))
    anomaly_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_anomaly}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                  dir_name=dir_name,
                                                                                  prefix_anomaly=prefix_anomaly,
                                                                                  id_name=id_name,
                                                                                  ext=ext)))
    anomaly_labels = numpy.ones(len(anomaly_files))
    files = numpy.concatenate((normal_files, anomaly_files), axis=0)
    labels = numpy.concatenate((normal_labels, anomaly_labels), axis=0)
    print("test_file  num : {num}".format(num=len(files)))
    if len(files) == 0:
        print("no_wav_file!!")
    print("\n========================================")

    return files, labels

In [ ]:
target_dir = "/content/drive/MyDrive/test/valve"

machine_type = os.path.split(target_dir)[1]
print("============== MODEL LOAD ==============")
# set model path
model_file = "/content/drive/MyDrive/models/valve/LSTM/model_valve.h5"

# load model file
if not os.path.exists(model_file):
  print("{} model not found ".format(machine_type))
  sys.exit(-1)
model = keras.models.load_model(model_file, custom_objects={'CustomModel': CustomModel, 'mse':mse_metric})
# model.summary()

machine_id_list = get_machine_id_list_for_test(target_dir)

# initialize lines in csv for AUC and pAUC
csv_lines = []

csv_lines.append([machine_type])
csv_lines.append(["id", "AUC", "pAUC"])
performance = []

for id_str in machine_id_list:
  # load test file

  id_num = id_str.split("_")[1]

  # Definizione della label "match" da utilizzare in fase di testing e del min e max da utilizzare per la normalizzazione
  # i min e max sono stati calcolati a partire dai dati di training.
  if id_num == "00":
    match_labels = numpy.asarray([1,0,0,0])
    mean = mean_00
    std = std_00
  if id_num == "02":
    match_labels = numpy.asarray([0,1,0,0])
    mean = mean_02
    std = std_02
  if id_num == "04":
    match_labels = numpy.asarray([0,0,1,0])
    mean = mean_04
    std = std_04
  if id_num == "06":
    match_labels = numpy.asarray([0,0,0,1])
    mean = mean_06
    std = std_06

  test_files, y_true = test_file_list_generator(target_dir, id_str)

  # setup anomaly score file path
  anomaly_score_csv = "/content/drive/MyDrive/models/valve/LSTM/anomaly_score_{machine_type}_{id_str}.csv".format(machine_type=machine_type, id_str=id_str)
  anomaly_score_list = []

  print("\n============== BEGIN TEST FOR A MACHINE ID {id} ==============".format(id=id_num))

  y_pred = [0. for k in test_files]

  for file_idx, file_path in tqdm(enumerate(test_files), total=len(test_files)):

    # Estrazione spettrogramma audio test
    data = file_to_vector_array(file_path, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=POWER)
    # Normalizzazione spettrogramma di test
    data = ( data - mean ) / (std)

    #print(data_aug_transpose.shape)

    data_aug = numpy.zeros((16, 128, 32))
    index = 0
    i = 0
    while (i+32) <= 313:
      vector_i = numpy.zeros((128,32))
      for j in range(0,128):
        vector_i[j] = data[j][i:i+32]
      data_aug[index] = vector_i
      index += 1
      i = i+18

    data_aug_transpose = numpy.zeros((len(data_aug), 32, 128))
    index = 0
    for elem in data_aug:
      data_aug_transpose[index] = elem.T
      index += 1
    #print(data_aug_transpose.shape)
    
    # Calcolo dell'errore medio sulle frame estratte dallo spettrogramma
    elem_error = []
    for elem in data_aug_transpose:
      predicted = model.predict([elem.reshape(1,32,128), match_labels.reshape(1,4)])
      errors = numpy.mean(numpy.square(elem - predicted), axis=1)
      elem_error.append(numpy.mean(errors))

    # Log dell'errore associato all'istanza di test
    y_pred[file_idx] = numpy.mean(errors)
    anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])
  
  save_csv(save_file_path=anomaly_score_csv, save_data=anomaly_score_list)
    
  # Calcolo AUC e pAUC per i dati con un certo ID_0x
  auc = metrics.roc_auc_score(y_true, y_pred)
  p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=0.1)
  csv_lines.append([id_str.split("_", 1)[1], auc, p_auc])
  performance.append([auc, p_auc])
  print("AUC : {}".format(auc))
  print("pAUC : {}".format(p_auc))

  print("\n============ END OF TEST FOR A MACHINE ID ============")

# Stampa di AUC e pAUC medi su tutti i dati di test (media di AUC e pAUC sui vari ID).
print("\n============ AVERAGE PERFORMANCES ============")
averaged_performance = numpy.mean(numpy.array(performance, dtype=float), axis=0)
csv_lines.append(["Average"] + list(averaged_performance))
csv_lines.append([])
print(averaged_performance)

result_path = "/content/drive/MyDrive/models/valve/LSTM/anomaly_score_avg.csv"
save_csv(save_file_path=result_path, save_data=csv_lines)

============== MODEL LOAD ==============


  0%|          | 0/219 [00:00<?, ?it/s]

target_dir : /content/drive/MyDrive/test/valve_id_00
test_file  num : 219


============== BEGIN TEST FOR A MACHINE ID 00 ==============


  0%|          | 0/220 [00:00<?, ?it/s]

AUC : 0.6588235294117647
pAUC : 0.6041574524546661

============ END OF TEST FOR A MACHINE ID ============
target_dir : /content/drive/MyDrive/test/valve_id_02
test_file  num : 220


============== BEGIN TEST FOR A MACHINE ID 02 ==============


 58%|█████▊    | 128/220 [01:30<01:07,  1.36it/s]